# Inferential Statistcs

추측 통계 

In [1]:
# 경고 메시지 출력 끄기
import warnings 
warnings.filterwarnings(action='ignore')

# 노트북 셀 표시를 브라우저 전체 폭 사용하기
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from IPython.display import clear_output

%matplotlib inline
import matplotlib.pyplot as plt

import os, sys, shutil, functools
import collections, pathlib, re, string

rseed = 22
import random
random.seed(rseed)

import numpy as np
np.random.seed(rseed)
np.set_printoptions(precision=5)
np.set_printoptions(formatter={'float_kind': "{:.5f}".format})

import pandas as pd
pd.set_option('display.max_rows', None) 
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_colwidth', None)
pd.options.display.float_format = '{:,.5f}'.format

import seaborn as sns

from pydataset import data

print(f"python ver={sys.version}")
print(f"pandas ver={pd.__version__}")
print(f"numpy ver={np.__version__}")

python ver=3.8.9 (default, Jun 27 2021, 02:41:12) 
[GCC 7.5.0]
pandas ver=1.2.5
numpy ver=1.19.5


## 추정 (Estimation)

기술 통계학에서는 분석 하고자 하는 모든 데이터의 통계적 특성을 통해 데이터를 분석하였습니다. 하지만, 현실에서는 시간/비용 등의 문제로 인하여 원하는 모든 데이터를 수집하여 분석할 수 없는 경우가 종종 발생합니다. 추정에서는 표본 데이터를 수집하여 해당 데이터를 통해 모집단의 통계적 특성을 추론함으로써 이런 문제점을 극복할 수 있습니다.

표본의 평균에서 모집단의 평균을 추정하는 것은 중심극한정리에 이론적 근간을 두고 있습니다. 

**중심극한정리(Central Limit Throrem)**

모집단(Population)이 정규분포하지 않아도 거기서 추출(sampling)한 표본(Sample)이 충분히(30 이상)크면 표본평균의 분포는 정규분포 한다.

표본 수: $n$, 모평균: $\mu$, 모분산: $\sigma^2$, 모표준편차: $\sigma$, 표본평균: $\bar{X}$, 표본분산: $s^2$, 표본표준편차: $s$

* 표본평균 $\bar{X}$ 의 분포 평균은 모집단의 평균 $\mu$ 과 일치
* 표본평균 $\bar{X}$ 의 분포 분산은 $\frac{\sigma^2}{n}$ (표준편차: $\frac{\sigma}{\sqrt{n}}$)
* 모집단의 분포가 어떠하든 표본 수 $n$ 가 커질수록 표본평균 $\bar{X}$ 의 분포는 정규분포

![](https://i.ibb.co/ZdZYZxW/central-limit-theorem.jpg)

https://www.youtube.com/watch?v=3SKwerKHbRk

### 점추정 (Point Estimation)

중심극한정리에 따라 모평균이나 모분산을 하나의 수치로 불편추정(unbiased estimation)합니다.  

* 모평균 점추정을 위한 불편평균(unbiased mean): $\hat{\mu} = \frac{1}{n}\sum{\bar{X}}$
* 모분산 점추정을 위한 불편분산(unbiased variance): $\hat{\sigma^2} = \frac{1}{n-1}\sum{(X-\bar{X})^2}$


In [73]:
from scipy import stats

# 모집단(Population): 평균 1.0 과 분산 2.0 의 정규분포
p_mean = 1.0
p_var = 2.0
# 정규분포 연속 확률 변수 (Normal continuous random variable)
rv = stats.norm(loc=p_mean, scale=np.sqrt(p_var))
print(f"population: p_mean={p_mean}, p_var={p_var}")

# 표본: 30개의 데이터를 가지는 표본을 10000개 랜덤 샘플링
n = 30
n_samples = 10000
# 무작위 표본, 랜덤 변량 (Random variates)
samples = rv.rvs((n, n_samples))
print(f"sample: n={n}, n_samples={n_samples}")
print(f"sample: samples.shape={samples.shape}")

# 모평균, 모분산의 불편 추정(unbiased estimation)
sample_means = np.mean(samples, axis=0)
# 모분산의 불편추정(unbiased estimation)을 위해서는 분산 계산시 자유도가 1 (n-1)이어야 함
ddof = 1 # Delta Degrees of Freedom (dof = n - ddof)
sample_vars = np.var(samples, axis=0, ddof=ddof) 
print(f"estimation: sample_means.shape={sample_means.shape}, sample_vars.shape={sample_vars.shape}")
s_mean = np.mean(sample_means)
s_var = np.mean(sample_vars)
print(f"estimation: p_mean={p_mean}, p_var={p_var}")
print(f"estimation: s_mean={s_mean:.2f}, s_var={s_var:.2f}")

population: p_mean=1.0, p_var=2.0
sample: n=30, n_samples=10000
sample: samples.shape=(30, 10000)
estimation: sample_means.shape=(10000,), sample_vars.shape=(10000,)
estimation: p_mean=1.0, p_var=2.0
estimation: s_mean=1.00, s_var=2.00


### 구간추정 (Interval Estimation)

모집단이 정규본포를 따른다는 가정하에 점추정도 좋은 추정량으로 의미가 있습니다만, 확률 변수이기 때문에 예상되는 오차를 표시해주는 것이 좀 더 정확한 추정이라고 할수 있습니다. 

중심극한 정리에 따라 표본평균의 분포는 정규분포를 따르기때문에 정규분포를 이용하면 모평균이 어느정도의 신뢰 구간(Confidence Interval)을 가지는지 추정이 가능하게 됩니다. 

![](https://www.spss-tutorials.com/img/confidence-interval-mean.png)

https://www.spss-tutorials.com/confidence-intervals/

95% 모평균의 신뢰 구간을 추청한다는 의미는 여러 표본을 만들경우 그 표본 중 95% 정도는 모평균을 포함한다는 의미입니다.

#### 모평균의 구간추정 

**모분산을 알고 있는 경우** 

신뢰 구간을 추청하는 방식을 이용하면 모분산($\sigma^2$)을 알고 있을 경우, 모평균의 95% 신뢰 레벨(Confidence Level) 또는 5% 유의 수준(Significance Level, $\alpha$)을 가지는 신뢰 구간(Confidence Interval)을 아래와 추정 할 수 있게 됩니다. 

$\mathsf{\text{Confidence Interval (z-score)} = \left[ \bar{x} - z_{\alpha/2} \times \frac{\sigma}{\sqrt{n}}, \bar{x}+z_{\alpha/2} \times \frac{\sigma}{\sqrt{n}} \right]}$

$\alpha/2$: Significance level for two-tailed, $\frac{\sigma}{\sqrt{n}}$: Standard error for population

$\bar{X} - z_{0.025} \times \frac{\sigma}{\sqrt{n}} \leq \mu \leq \bar{X} + z_{0.025} \times \frac{\sigma}{\sqrt{n}}$

$\bar{X} - 1.96 \times \frac{\sigma}{\sqrt{n}} \leq \mu \leq \bar{X} + 1.96 \times \frac{\sigma}{\sqrt{n}}$

In [74]:
from scipy import stats

# 모집단(Population): 평균 1.0 과 분산 2.0 의 정규분포
p_mean = 1.0
p_var = 2.0
p_std = np.sqrt(p_var)
# 정규분포 연속 확률 변수 (Normal continuous random variable)
rv = stats.norm(loc=p_mean, scale=np.sqrt(p_var))
print(f"population: p_mean={p_mean}, p_var={p_var}")

# 표본: 30개의 데이터를 가지는 표본을 10000개 랜덤 샘플링
n = 30
n_samples = 10000
# 무작위 표본, 랜덤 변량 (Random variates)
samples = rv.rvs((n, n_samples))
print(f"sample: n={n}, n_samples={n_samples}")
print(f"sample: samples.shape={samples.shape}")

# 신뢰구간 (confidence level), 유의수준 (significance level)
cl = .95
sl = (1.-cl)/2 # for Two-Tailed (양측검정) 
print(f"estimation: cl={cl}, sl={sl:.3f}")

# 표본평균(Sample Mean), 표본편차(Sample Standard Deviation) , 표준오차 (Standard Error)
sample_means = np.mean(samples, axis=0)
s_mean = np.mean(sample_means)
p_ste = p_std / np.sqrt(n)
# 신뢰구간 하한 (Lower confidence limit), 신뢰구간 상한 (Upper confidence limit)
# 경계값 (Critical Value)
z_cv = stats.norm.ppf(1.-sl)
lcl = s_mean - z_cv * p_ste
ucl = s_mean + z_cv * p_ste
print(f"estimation: lcl={lcl:.5f} <= p_mean={p_mean} <= ucl={ucl:.5f}")

# interval 함수 이용 
lcl, ucl = stats.norm.interval(alpha=cl, loc=s_mean, scale=p_ste) 
print(f"estimation: lcl={lcl:.5f} <= p_mean={p_mean} <= ucl={ucl:.5f}")

population: p_mean=1.0, p_var=2.0
sample: n=30, n_samples=10000
sample: samples.shape=(30, 10000)
estimation: cl=0.95, sl=0.025
estimation: lcl=0.49349 <= p_mean=1.0 <= ucl=1.50561
estimation: lcl=0.49349 <= p_mean=1.0 <= ucl=1.50561


**모분산을 모르는 경우**

하지만, 일반적으로 모집단의 분산을 모르기때문에, 모집단이 정규분포를 따른다른 가정하에 t 분포 및 표본분산($s^2$)를 이용하여 모평균의 신뢰 구간을 추정할 수 있습니다. t 분포는 표본 수가 적어도 사용 가능하며, 표본 수가 충분히(30 이상) 크면 정규 분포와 거의 같아지므로 작은 표본에서도 사용할 수 있습니다. t 분포에서 95% 신뢰 구간은 아래와 같이 추정할 수 있게 됩니다.

$\mathsf{\text{Confidence Interval (t-score)} = \left[ \bar{x} - t_{\alpha/2}(n-1) \times \frac{s}{\sqrt{n}}, \bar{x}+t_{\alpha/2}(n-1) \times \frac{s}{\sqrt{n}} \right]}$

$\alpha/2$: Significance level for two-tailed, $\frac{s}{\sqrt{n}}$: Standard error for sample

$\bar{X} - t_{0.025}(n-1) \times \frac{s}{\sqrt{n}} \leq \mu \leq \bar{X} + t_{0.025}(n-1) \times \frac{s}{\sqrt{n}}$

t 분포의 경우 표본 수가 확률에 영향을 미치므로, 예로 n = 10 이라고 한다면, 아래와 같습니다. 

$\bar{X} - 2.26 \times \frac{s}{\sqrt{n}} \leq \mu \leq \bar{X} + 2.26 \times \frac{s}{\sqrt{n}}$

In [80]:
from scipy import stats

# 모집단(Population): 평균 1.0 과 분산 2.0 의 정규분포
p_mean = 1.0
p_var = 2.0
# 정규분포 연속 확률 변수 (Normal continuous random variable)
rv = stats.norm(loc=p_mean, scale=np.sqrt(p_var))
print(f"population: p_mean={p_mean}, p_var={p_var}")

# 표본: 10개의 데이터를 가지는 표본을 10000개 랜덤 샘플링
n = 10
n_samples = 10000
# 무작위 표본, 랜덤 변량 (Random variates)
samples = rv.rvs((n, n_samples))
print(f"sample: n={n}, n_samples={n_samples}")
print(f"sample: samples.shape={samples.shape}")

# 신뢰구간 (confidence level), 유의수준 (significance level), 자유도 (Degree of Freedom)
cl = .95
sl = (1.-cl)/2 # for Two-Tailed (양측검정) 
dof = n - 1 # Degree of Freedom
ddof = 1 # Delta Degrees of Freedom (dof = n - ddof)
print(f"estimation: cl={cl}, sl={sl:.3f}, ddof={ddof}")

# 표본평균(Sample Mean), 표본편차(Sample Standard Deviation) , 표준오차 (Standard Error)
sample_means = np.mean(samples, axis=0)
s_mean = np.mean(sample_means)
s_std = np.std(sample_means) 
s_ste = s_std / np.sqrt(n)
# 신뢰구간 하한 (Lower confidence limit), 신뢰구간 상한 (Upper confidence limit)
# 경계값 (Critical Value)
t_cv = abs(stats.t.ppf(sl, dof))
lcl = s_mean - t_cv * s_ste
ucl = s_mean + t_cv * s_ste
print(f"estimation: lcl={lcl:.5f} <= p_mean={p_mean} <= ucl={ucl:.5f}")

# interval 함수 이용 
lcl, ucl = stats.t.interval(alpha=cl, df=dof, loc=s_mean, scale=s_ste) 
print(f"estimation: lcl={lcl:.5f} <= p_mean={p_mean} <= ucl={ucl:.5f}")

population: p_mean=1.0, p_var=2.0
sample: n=10, n_samples=10000
sample: samples.shape=(10, 10000)
estimation: cl=0.95, sl=0.025, ddof=1
estimation: lcl=0.68179 <= p_mean=1.0 <= ucl=1.32304
estimation: lcl=0.68179 <= p_mean=1.0 <= ucl=1.32304


## 가설검정 (Testing Hypothesis)

가설검정이란 모집단의 통계량에 대해 두가지 가설을 세우고 표본의 통계량을 이용하여 어느 가설이 옳은지 판단하는 통계적 기법입니다. 모집단 통계량에 대해 귀무가설(H0, Null hypothesis, 기각하고 싶은 가설)을 세우고 이에 대한 표본을 수집하여 계산한 표본통계량(검정통계량)이 확률적으로 거의 일어나지 않을 일임을 증명하여 귀무가설을 기각하고, 대립가설(H1, Alternative hypothesis, 채택하고 싶은 가설)을 채택하는 방식입니다.

